In [ ]:
!nvidia-smi

Tue Nov 15 05:22:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
import math
from gensim.models import FastText
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import pickle
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import re
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

!pip install wikipedia
import wikipedia

nltk.download('treebank')
from nltk.corpus import treebank
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=a79006438f6c40e4f02502b18b527404c7eaf0b867dce3ba4288295c654535eb
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [ ]:
from google.colab import drive 
drive.mount('/content/drive/') 

Mounted at /content/drive/


# Load

In [ ]:
import numpy as np
import pandas as pd
import pickle
path='/content/drive/MyDrive/CONDA/data/'
file_name=('train_new.pkl', 'valid_new.pkl', 'test_new.pkl')
with open(file=path+file_name[0], mode='rb') as f:
    train_data=pickle.load(f)
with open(file=path+file_name[1], mode='rb') as f:
    valid_data=pickle.load(f)
with open(file=path+file_name[2], mode='rb') as f:
    test_data=pickle.load(f)

In [ ]:
training_data = pd.read_csv("/content/drive/MyDrive/NER/CONDA_train_leaderboard.csv")
testing_data = pd.read_csv("/content/drive/MyDrive/NER/CONDA_valid_leaderboard.csv")

training_post=training_data['tokenized'].tolist()
training_labels=training_data['slotClasses'].tolist()
training_int = training_data['intentClass'].tolist()

testing_post=testing_data['tokenized'].tolist()
testing_labels=testing_data['slotClasses'].tolist()
testing_int = testing_data['intentClass'].tolist()

truelabels = pd.read_csv("/content/drive/MyDrive/NER/CONDA_test_leaderboard.csv")
truelabels = truelabels['tokenized'].tolist()

In [ ]:
def remove_punctuation(data):
    processed_data = []
    for string in data:
        string = string.replace('[', '')
        string = string.replace(']', '')
        string = string.replace('!', '')
        string = string.replace('#', '')
        processed_data.append(string.strip().lower())
    return processed_data
training_post = remove_punctuation(training_post)
testing_post = remove_punctuation(testing_post)
truelabels = remove_punctuation(truelabels)


def tokenization(sentences):
    ls = []
    for sent in sentences:
        sent = sent.split()
        ls.append(sent)
    return ls

training_post = tokenization(training_post)
testing_post = tokenization(testing_post)
truelabels = tokenization(truelabels)

In [ ]:
training_labels = tokenization(training_labels)
testing_labels = tokenization(testing_labels)

In [ ]:
train_data = pd.DataFrame(np.array([training_post+testing_post[:5000],training_labels+testing_labels[:5000]]),
                          index=['lower_token', 'slot_label']).T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
valid_data = pd.DataFrame(np.array([testing_post[5000:],testing_labels[5000:]]),
                          index=['lower_token', 'slot_label']).T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
test_data = pd.DataFrame(np.array([truelabels]),
                          index=['lower_token']).T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
train_data

,lower_token,slot_label
0,[wow],[O]
1,[wtf],[T]
2,"[wpe, wpe]","[O, O]"
3,[hahaha],[O]
4,[wtf],[T]
...,...,...
31081,"[passsion, over]","[O, O]"
31082,[ya],[O]
31083,"[just, end, it]","[O, O, P]"
31084,[hahahahah],[O]


In [ ]:
all_sentences=train_data.lower_token.tolist()+valid_data.lower_token.tolist()+test_data.lower_token.tolist()

In [ ]:
max_length=0
for i in all_sentences:
    max_length=max(max_length, len(i))
max_length

72

In [ ]:
#Get word_list and word_dict
def construct_wordlist_and_worddict(all_sentences):
    word_set = set() 
    for sentence in all_sentences:
        for word in sentence:
            word_set.add(word)
    word_list = list(word_set) 
    word_list.sort()
    word_list.append('[PAD]')
    word_list.append('[UNKNOWN]')

    #Get Word Dict
    word_dict = dict()
    ind = 0
    for word in word_list:
        word_dict[word] = ind
        ind += 1
    return word_list, word_dict


#call
word_list, word_dict = construct_wordlist_and_worddict(all_sentences)

In [ ]:
word_dict

In [ ]:
def encode_and_add_padding(sentences, word_dict, max_length):
    sentence_encode = []
    for sentence in sentences:
        temp_encoded = [word_dict[word] if word in word_dict  #If the word is in word_dict, return the corresponding value
                        else word_dict['[UNKNOWN]'] #If the word is not in word_dict, then the value corresponding to [UNKNOWN] is returned
                        for word in sentence] 
        # if len(temp_encoded) < max_length: #If shorter than the predetermined length, the value of [PAD] will be added
        #     temp_encoded += [word_dict['[PAD]']] * (max_length - len(temp_encoded))
        sentence_encode.append(temp_encoded)
    return sentence_encode 

#call
train_encode=encode_and_add_padding(train_data.lower_token.tolist(), word_dict, max_length)
valid_encode=encode_and_add_padding(valid_data.lower_token.tolist(), word_dict, max_length)
test_encode=encode_and_add_padding(test_data.lower_token.tolist(), word_dict, max_length)

In [ ]:
train_data.insert(train_data.shape[-1], 'sentence_encode', train_encode)
valid_data.insert(valid_data.shape[-1], 'sentence_encode', valid_encode)
test_data.insert(test_data.shape[-1], 'sentence_encode', test_encode)

# Sematic Embedding

In [ ]:
from gensim.models import FastText
sem_model = FastText(sentences = all_sentences, size=50, window=5, min_count=1, workers=4, sg=1)

In [ ]:
def construct_embedding_tabel(word_dict, model ):
    emb_table=[]
    emb_dim = model.vector_size
    count=0
    for word in word_dict.keys():
        if model.wv.__contains__(word):
            emb_table.append(model.wv.__getitem__(word)) #Copy the vector corresponding to word_list from trained embedding into emb_table
        else: 
            count+=1
            emb_table.append([0]*emb_dim) #only for [UNKNOWN]
    emb_table = np.array(emb_table)
    return emb_table, count

#call
sem_emb_table, count=construct_embedding_tabel(word_dict, sem_model)

In [ ]:
sem_emb_table.shape

(12765, 50)

# Domain Embedding

## corpus

In [ ]:
wiki_dota_doc = wikipedia.page("DOTA2").content
wiki_lol_doc = wikipedia.page("League of Legends").content
wiki_video_doc = wikipedia.page("Video Game").content
wiki_moba_doc = wikipedia.page("Multiplayer online battle arena").content

In [ ]:
# Source: https://dota2.fandom.com/wiki/Glossary
dota_slang_doc = """AA Initialism for Ancient Apparition. AC Initialism for Assault Cuirass. Aggro Abbreviation for aggression. The programmed aggression of AI controlled towers and creeps. Refers to getting the attention of a particular hostile unit, e.g. "I have aggro" means a hostile unit is focusing on attacking you.
AM Initialism for Anti-Mage. AoE  Initialism for Area of Effect. It refers to a spell, attack, or effect that affects an area around a point (for example, Dragon Slave is an "AoE spell"). Also used to refer to the range or size of one such spell/attack/effect, for example Reverse Polarity affects enemies in a 410 radius, therefore it has an AoE of 410, or "410 AoE". AW 
Initialism for Arc Warden.
B 
Initialism for "Back" or "Get Back". Used to call for a retreat. A player saying "b" is usually suggesting that everyone back up or run away.
BAT 
Abbreviation for Base Attack Time, which determines how long an unbuffed unit with 0 agility and 0% bonus attack speed has to wait to attack again. Lower BAT units are able to attack more frequently. Most heroes have a BAT of 1.70. It is considered better to have a lower BAT and worse to have a higher BAT.
Backdoor 
The act of attacking enemy structures while a player's own lane creeps have not reached the structure yet. Usually ineffective due to backdoor protection.
Ball 
May refer to Io or snowballing or close positioning of teammates when pushing highground ('don't ball for ravage')
Bara, Barathum 
Refers to Spirit Breaker's name as he was commonly referenced in Dota 1.
Barracks
Structures found in each lane that when destroyed permanently grant super creeps to the team that destroyed it.
Basi 
refers to Ring of Basilius
BB 
Initialism for Bristleback or Buyback.
BD 
Initialism for backdooring.
BF 
Initialism for Battle Fury, also often referred to as "bfury", or Butterfly.
BH 
Initialism for either Bounty Hunter, or Enigma's ultimate, Black Hole.
Bird 
May refer to Phoenix, Skywrath Mage (and his spells), Visage's Familiars, or the flying courier.
BKB 
Initialism for Black King Bar.
Blocking 
Refers to one of the following: the act of "blocking" creeps with your hero's body as they run down the lane, impeding their movement so that the creeps meet closer toward your tower; the act of deliberately and repeatedly moving directly in front of an enemy hero (in the same manner as creep blocking) to impede their movement, often to help allies attack them; the act of "blocking" a neutral camp with a unit or a ward -- because if there is a unit or ward in the spawn box at X:00 on the clock, the camp will not spawn new creeps.
BM 
Initialism for Beastmaster, Brewmaster, Broodmother, Blade Mail or "bad manners".
Bot 
Refers to either the bottom lane of the map, or an AI-controlled hero.
BoT, BoTs
Initialism for Boots of Travel.
Book 
Refers to Necronomicon, short for "necro book". Can also be referred to as book1, book2, or book3 based on its level.
Bottle Crow 
The tactic of giving one's Bottle to the courier, then sending it on a round trip back to base to refill the bottle and give it back to the owner. This keeps the player's HP/MP healthy, and they do not have to miss any exp/gold from lane refilling it themselves. Note that any bottle that is not full slows the courier by 30%, and the act of bottle crowing may hamper player's allies as it gets harder to deliver their own items while the courier is making trips. (no longer possible, patched out)
BRB 
Initialism for "be right back".
Break 
Refers to either the effect of disabling the effects of a hero's passive abilities or the other passive ability of Tranquil Boots.
BS 
Initialism for Bloodseeker or Bloodstone. May also be used as an acronym for "bullshit".
Buff 
A beneficial spell or effect placed on a unit. Refers to the opposite of a debuff.
Burst or Burst Damage 
Refers to a high amount of damage dealt over a short period of time. Usually involves nukes.
Carry 
A type of hero which can overpower the enemy team in the late game. These heroes tend to scale very well with gold and experience, and thus require large sums of it in order to be successful.
Caster 
Either refers to a hero whose primary function is to cast spells prolifically, or a live person who commentates on an ongoing game.
CC 
Initialism for Crowd Control. See disable.
CD 
Initialism for cooldown or Captain's Draft.
CDR 
Initialism for cooldown reduction.
Chicken or Chick 
Refers to Animal Courier.
Chieftain 
See Tauren Chieftain.
CK 
Initialism for Chaos Knight. May also refer to "creep kill" (see CS or last hit).
CM 
Initialism for Crystal Maiden or Captain's Mode.
Comeback 
A situation when a team is performing poorly for most of the game, but has managed to fight back, turn the tide of battle and ultimately win the game.
Crow 
Refers to Flying Courier, which upgrades a walking courier. A player asking to "please crow it" is asking for someone to upgrade the courier.
CS 
Initialism for Creep Score, the amount of last hits a player has, or the amount of last hits and denies a player has. In DotA 1, typing -cs would show the player's last hits and denies count. May also refer to "creep steal", the act of stealing last hits from an allied player.
CW 
Initialism for Centaur Warrunner or Clockwerk.
DD 
Initialism for Rune of Double Damage.
Debuff 
A detrimental spell of effect placed on a unit. Refers to the opposite of a buff.
Deny 
Killing your allied unit, hero or building, in order to prevent an enemy from gaining the gold and experience it gives. You can manually attack your own creeps if their health is below 50%, your own towers below if they are below 10%, and heroes affected by certain DoTs. A hero can also be said to be denied if they die to neutral creeps instead of to an enemy unit.
Deward 
The act of removing enemy wards, usually mostly pertaining to Observer Wards.
DGU 
Initialism for "Don't give up".
Dieback 
When a player dies after using their Buyback, resulting in a much longer respawn time and them unable to use it once more as it will most definitely be on cooldown.
Disable 
A catch-all term referring to any spell, ability, or effect that otherwise prevents an enemy hero from moving, casting, and/or attacking, leaving them helpless for a short period of time or in general impeding their ability to act. See the main article for more information.
Dive 
The act of running at a hero that is within their tower range in order to attempt to secure a kill. This can sometimes be a risky play in the early stages of the game when tower damage output is more significant. Dives are usually attempted by more than one hero, but can also be done by a single hero. Also referred to as a "tower dive".
DK 
Initialism for Dragon Knight.
DoT 
Damage over Time, an applied effect that repeatedly inflicts damage at a regular interval for a specific duration. A classic example would be Venomancer's Poison Nova.
DP 
Initialism for Death Prophet.
DPS 
Initialism for damage per second, a measure of the damage dealt by a hero or unit over one second. Usually refers to heroes with consistent damage output over a period rather than heroes with high burst damage periodically.
Dunk 
Refers to either Axe's ultimate, Culling Blade, or Earthshaker's ultimate, Echo Slam.
Durable 
A hero that can take a lot of damage and abuse before dying. See the main article for more information.
Dust 
Abbreviation for the item, Dust of Appearance.
Easy lane
See Safe Lane.
Egg 
Refers to Phoenix' ultimate, Supernova.
Epi 
Refers to Sand King's ultimate, Epicenter.
ES 
Initialism for Earthshaker, Earth Spirit, or Ember Spirit.
ET 
Initialism for Elder Titan.
Exp or XP 
Experience, the resource used by heroes to gain levels, level skills, and otherwise increase in power over the course of a game.
ez 
"Easy", typically said to mock the enemy team's lesser power.
F - J
Fade time 
The amount of time it takes for a unit to become completely invisible following the activating of an invisibility effect.
Farm or Farming 
The process of steadily earning gold and experience by killing lane creeps and/or neutral creeps. This tactic is often slow and tedious, but is usually necessary for Carries to reach their full potential.
FB 
Initialism for First Blood.
Feed 
The act of "feeding" gold and experience to the enemy team by dying repeatedly. Can be intentional or unintentional. Those who feed are called "feeders".
Flash Farming 
A style of farming when you farm your lane creeps until the enemy tower and then proceed to clear neutral creep camps in quick and efficient rotations. This is often the fastest method of farming, but it also allows the enemy to farm your creeps freely by their tower and can be risky, as being near the enemy tower is often an easy gank, or the enemy might know exactly where you are clearing the camp closest to the lane tanking creeps and losing health, etc.
Flash Farming Skill/Ability
An AoE ability that facilitates flash farming by allowing a hero to quickly clear lane creeps or groups of neutrals. The classic flash farming ability is Shadow Fiend's Shadowraze which allows him to mow down wave after wave of creeps with ease. It is common to hit lane creeps below threshold and then clear them all in one shot with the flash farming ability, and then go off to check rune, kill neutrals, etc.
FF 
According to DotA history, "ff" appeared on DotA-League.com (at this time it was for DotA 1), and it means "forfeit". You had to type "-ff" in the game so it can detected and recorded by the dota-league plugin and save the data to your dota-league profile. If all 5 members of one team type it the game would end. Now replaced by calling "gg" if the surrender option is enabled in a lobby. More commonly used as abbreviation to "finish fast", asking enemy team to end the game that's already unwinnable for other side.
Fortification/Fortify 
The Glyph of Fortification is the button in the bottom right hand of the screen that renders all allied buildings invulnerable for 5 seconds on a 5 minute cooldown. If a player says "don't fort", they are telling the team to save the glyph for later. Towers are "fortified" for the 5 seconds that the glyph is active.
FoW 
Initialism for Fog of War. Refers to the portion of the map that is dark and unseen. If you cannot see an area, then it is said to be "fogged". Most of the map is "fogged" by default, such as Roshan's lair, and the enemy base.
Furion 
Nature's Prophet's name in WC3 DotA. Still referenced in the game's files for the hero.
Gank 
Abbreviation for Gang Kill, but over time the term has come to refer to any time a hero or group of heroes attempts to pick off an enemy hero (or enemy heroes) by surprise, but usually with superior forces (such as 2v1).
Gem 
The Gem of True Sight. Most commonly means to either buy a gem, or to be careful of the enemy team as they possess a gem (especially for allied invisible heroes).
gg 
"Good game". Said by either team when they are winning or losing and the match is close to ending, or when a team or a player gives up or claims victory. For example, both teams might say "gg" to congratulate each other after a hard fought match, or a team might call out "gg" after their ally abandons in an assumption that they can no longer win.
ggwp 
"Good game, well played". Used in a similar fashion as "gg", but also congratulates the enemy for skillful play.
gj 
"Good job", typically said to compliment an ally.
glhf 
"Good luck, have fun". Usually said at the start of a game to encourage other players, or just to be courteous.
Golem 
Refers to Mud Golems, Ancient Golems, or Warlock's ultimate summon through Chaotic Offering.
Gs 
Initialism for Grimstroke.
Guinsoo 
A reference to Scythe of Vyse's full name in WC3 DotA, which in turn references one of the developers of the original DotA maps.
Hard Carry 
A type of carry hero which scales incredibly well with items and requires a substantial amount of farm to be effective. Examples of this are Anti-Mage and Spectre. Hard carries are usually weaker than other semi-carry or non-carry heroes at the early-mid stages of the game, when they do not yet have their core items.
Hard Lane 
The lane of either faction where the creep wave meets up furthest from the tower, making those who lane in it more susceptible to ganks (top lane for Radiant, bottom lane for Dire). See Lane.
HH 
Initialism for the item Heaven's Halberd.
Hook 
Refers to either Pudge's signature ability, Meat Hook, or Clockwerk's ultimate, Hookshot.
HP 
Health/hit points, referring to the health a unit has.
inc 
Abbreviation for "incoming", usually referring to enemy players heading to a lane to gank.
Initiation 
The act of starting (initiating) a teamfight. A good initiation can catch the enemy team off-guard, possibly losing quickly to the attacking team. A hero adept at performing this is called an initiator.
Janggo 
Drums of Endurance's former name in WC3 DotA.
Juking 
Running and weaving around trees, fog and other obstacles in such a manner to avoid and possibly escape an enemy.
Jungling 
The process of killing the neutral creeps in the woods (aka "jungle") between the lanes. Killing ancient creeps also counts as jungling, but is more often referred to as ancienting or "clearing ancients". A hero adept at jungling (be it by having summons to absorb the creeps' attacks or having good durability/self-sustain) is termed a jungler.
K - O
Kick 
More commonly used to refer to Earth Spirit's Boulder Smash. May also refer to removing a player from a lobby or game or Tusk's Walrus Kick.
Kite or Kiting 
See pulling. A technique where a hero gets the attention of a hostile unit to draw them away or force them to follow. Can also be used to refer to a period of repeated hit-and-run attacks where the target is kept out of range.
KotL 
Initialism for Keeper of the Light.
KS 
Either means "kill steal" or, usually jokingly, "kill secure". Firstly, it refers to the act of one hero stealing the kill of another hero that could have easily taken it themselves. Secondly, it refers to when another hero "secures" the kill if the carry or another hero could not do so themselves. The latter meaning is also sometimes interchangeably used with the former meaning in a satirical or comedic way.
LC 
Initialism for Legion Commander.
Leoric 
Wraith King's name in WC3 Dota.
Long lane 
See Hard Lane.
Lothar's or Lothar's Edge 
Shadow Blade's name in WC3 DotA.
LS 
Initialism for Lifestealer.
LSA 
Initialism for Lina's ability, Light Strike Array.
Manfight 
Refers to when two heroes, usually melee, continuously attack one another without kiting (see above) until one hero either dies or retreats. Often implies a 1v1 battle without allies of either hero nearby. Heroes such as Ursa and Troll Warlord excel at manfights.
Meatball 
Refers to Invoker's Chaos Meteor.
Micro 
Micromanagement. Refers to (effective) control and usage of multiple units, items, and abilities in quick succession. Bad micromanagement can result in the many units becoming a hindrance or a liability rather than an asset or ability combos being wasted and/or used in the wrong situations, while a good one can easily overpower an enemy in seconds. Meepo, Invoker, and Chen are good examples of heroes who require good "micro".
Mid/Middle 
The middle of the three lanes in the map.
Miss, Missing, or MIA 
Mentions that a particular hero has gone absent (missing) from their lane, and is probably setting up for a gank. Warning missing enemy heroes is crucial to warn your allies of a possible ambush in any of the lanes. "MIA" is an acronym for "Missing In Action".
MK 
Initialism for Monkey King.
MKB 
Initialism for Monkey King Bar.
MP 
Mana points, referring to the amount of mana a unit has.
MS 
Initialism for Movement Speed, or sometimes "missing".
Mute 
Refers either to the effect that prevents item usage, or blocking any means of communication from a player in the game to avoid harassment.
N'aix Bomb 
Refers to when Lifestealer uses his ultimate, Infest, on an allied hero, preferably one with high mobility or with a Blink Dagger (and perhaps a disable), so that Lifestealer bursts out of his ally upon arrival, dealing additional damage and having two heroes down their target.
Necro 
May refer to Necrophos or the Necronomicon. Rarely used to refer to Visage, the Bound Form of Necro'lic.
Nerubian/Nerub 
Refers to Nyx Assassin and Weaver's former titles in WC3 DotA (the Nerubian Assassin and Nerubian Weaver, respectively).
nj 
"Nice job", analogous to gj.
NP 
Initialism for Nature's Prophet or "no problem".
Nuke 
A spell whose purpose is to deal a large amount of damage immediately or in a very short span of time. Heroes adept at nuking are referred to as nukers.
OC 
Initialism for Octarine Core.
OD 
Initialism for Outworld Destroyer.
Offlane 
See Hard Lane.
Offlaner 
A hero sent down to the offlane/hard lane, usually on their own. These heroes usually have good escape abilities or are naturally durable to withstand being alone and facing usually two enemy heroes (and being more vulnerable to ganks).
Omni 
Refers to Omniknight or Juggernaut's ultimate, Omnislash.
OoM 
Initialism for "out of mana".
OoV 
Initialism for Orb of Venom.
Orb effect 
The name used for unique attack modifiers in WC3 DotA.
Orb walk 
Using your orb effect (aka autocast) ability, i.e. Frost Arrows, Searing Arrows, to harass the enemy while moving forward to gain distance on the target and also preventing enemy creep aggro.
P - T
P 
Abbreviation for "push" or "pause". Players saying "p" means they are either suggesting the team to push an enemy lane or asking for a pause.
PA 
Initialism for Phantom Assassin.
Panda/Pandaren 
Brewmaster's former title in WC3 DotA.
Pet 
A creature that a hero can summon or convert to their side. For example, the Eidolons are Enigma's pets. Could also refer to the cosmetic pets that follow your hero in-game.
Pit Lord 
Underlord's former title in WC3 DotA.
PL 
Initialism for Phantom Lancer.
PP 
Initialism for "pause please".
PotM 
Initialism for Mirana, the Princess of the Moon.
Proc 
Short for "Programmed Random Occurence". It refers to the triggering of effects, whether the occurence is random (such as Wraith King's critical strikes on attack) or regular (such as Bristleback's quill spray every 250 damage he takes). When these effects trigger or "process", they are said to "proc". The rate of occurence for random effects is termed the "Proc rate".
Pull or Pulling
A technique where a hero gets the attention of a hostile unit to draw them away or force them to follow. This more usually refers to "creep pulling", which involves pulling lane creeps away from their lane by aggroing nearby creep camps into the chosen lane, which attracts the lane creeps and forces them to fight the aggroed creep camp for some time, which in turn makes the enemy creeps push forward, closer to the player's tower, letting them (or their carry) gain gold and experience underneath the safety of their tower.
QoP 
Initialism for Queen of Pain.
Rat 
A strategy in which heroes avoid 5-man teamfights, and focus on pushing other lanes instead.
Rax 
Abbreviation for barracks.
Recrow 
To reuse the courier after it has finished performing one of its deliveries/tasks. Since the courier can only perform one task for one player at a time, this is mentioned to remind/alert other players that the courier is now free to use.
Reuse 
See recrow, usually mentioned by Southeast Asian players.
Ricer 
Hardcore farmer whose main goal is to be extremely strengthened by the time they come out of their farming.
Ring or RoB/RoH/RoR/RoT 
Refers to Ring of Basilius, Ring of Health, Ring of Regeneration, and Ring of Tarrasque respectively. "Ring" usually refers to one of these, based on context.
RNG 
Random Number Generation, referring to the proc chance of abilities and items such as Bash or Evasion.
Ro3 
See Roshan, usually mentioned by Chinese players.
Roamer 
A hero that jumps between lanes, especially in the early game, in order to gank enemy heroes or defend allies.
Rock 
May refer to Warlock's ultimate: Chaotic Offering, Earth Spirit's Stone Remnants, the Ancient Rock and Granite Golems, or the Mud Golem's ability: Hurl Boulder.
Rosh or RS 
Roshan is a difficult-to-kill neutral creep that drops the Aegis of the Immortal when killed (or Refresher Shard on his third death and afterward). Spawns at the beginning of the game, respawns 8-11 minutes after he is killed.
RP 
Magnus' ultimate, Reverse Polarity.
Safe lane 
The lane of either faction, where the creep wave meets closest to the tower, making farming easier and less risky for those in the lane (bottom lane for Radiant, top lane for Dire). See Lane.
SB 
Initialism for Spirit Breaker or Shadow Blade.
Scepter 
Mostly refers to Aghanim's Scepter, although can also refer to Eul's Scepter of Divinity or Ghost Scepter.
SD 
Initialism for Shadow Demon or "self denial" (see suicide).
SF 
Initialism for Shadow Fiend.
Sheepstick 
Refers to the Scythe of Vyse, since back in WC3 DotA, it actually turned the target into a sheep (unlike in Dota 2, where the hex model is a pig).
Short lane
See Safe Lane.
Silence 
The effect of preventing a unit from casting spells, but can still allows item usage and passive abilities are unaffected. See the main article for more information.
SK 
Initialism for Sand King. May also refer to Wraith King in reference to his former title, Skeleton King, or Terrorblade, in reference to his former title Soul Keeper.
Skillshot 
An ability that requires proper aim and timing to hit an enemy. Some examples would be Sacred Arrow and Powershot.
Smoke 
Refers to either Smoke of Deceit, or Riki's ability Smoke Screen.
Snowballing 
The situation where a hero or team just gets stronger as the games goes on (usually through getting kills) to the point where it is very difficult to stop them, much like a snowball rolling down a hill getting bigger.
SnY 
Initialism for Sange and Yasha.
SOD 
Initialism for Smoke of Deceit.
Solo 
Being the only hero on the lane. A player may call for "solo" to prevent other players from interrupting their lane and splitting experience and gold.
Soul Keeper 
Terrorblade's former title in WC3 DotA.
Spirit Hero 
Shorthand for Ember Spirit, Storm Spirit, and Void Spirit. These three heroes share many attributes, such as typically being played mid, and having versatile kits featuring heavy burst damage and extremely reliable long-range engagement/escape tools. Earth Spirit is generally not included in this definition, as he is more often played as a support.
Squishy 
A hero that can only take relatively little damage or abuse before dying.
SR 
Initialism for status resistance.
SS
See missing. Mentions that a particular hero has gone missing and is probably setting up for a gank, e.g. "Earthshaker ss" means that Earthshaker isn't visible to the team. Also refers to the ultimate of a hero ("special/super skill"), usually mentioned by Southeast Asian players. May also refer to Storm Spirit, Shadow Shaman or Sun Strike.
Stacking
May also be referred to as "creep stacking", it is a technique where a creep camp is aggroed at a certain time before neutral spawning, pulling them away from their camp's area and allowing a new set of creeps to spawn. This can be done multiple times, usually up to three at maximum, then letting their team's carry farm the stacked creeps for a massive gold and experience gain.
Static Farming
A sub-type of farming where you aim to only kill enemy lane creeps when they have low health and deny the allied lane creeps whenever possible, with the goal of maintaining an equilibrium of the lane creeps in a safe position. This is often a slower method of farming, but allows you to farm safely with little risk of dying to a gank.
Stygian
References the Desolator's former full name in WC3 DotA.
Suicide
The act of killing oneself to deny the enemy of gold and experience. May also directly refer to Techies' ability Blast Off! (formerly Suicide Squad, Attack!).
Suicide lane
See Hard Lane.
Summoner
A hero whose primary function is summoning or controlling creeps.
Support
A hero whose primary function is to help their own team through heals, buffs, and detection, or sabotaging the enemy team through disables, slows, or debuffs.
TA 
Initialism for Templar Assassin.
Tank 
See Durable.
Tauren Chieftain (or simply Tauren) 
Elder Titan's former title in WC3 DotA.
TB 
Initialism for Terrorblade.
Teamwipe 
The act of an entire team dying at once.
THD 
Twin Head Dragon, the title of Jakiro.
Throw 
The act of giving a supreme advantage to the supposedly losing team, letting them win the match instead. This can be intentional or unintentional.
Throne 
See Ancient. Remnant of WC3 DOTA in which the Scourge's (Dire) Ancient is using the Frozen Throne model.
Top 
Refers to the top lane of the map. For the Radiant, this is the lane to the North. For the Dire, this is the lane to the West.
TP 
Initialism for Teleport or Town Portal Scroll.
True Sight 
Anything invisible caught in the radius of a True Sight source gets revealed. Towers, a Gem of True Sight, Sentry Wards, Dust of Appearance, and some abilities of certain heroes (such as Slardar's Amplify Damage and Zeus' Lightning Bolt) all provide True Sight. See the main article for more information.
Turnaround 
The act of a team facing the enemy in a situation where they might be expected to run, such as when a team initiates on the other. Instead of running away, they try to turn the battle around and fight back, possibly even winning the battle themselves.
U - Z
UAM 
Initialism for unique attack modifier.
Ult or Ulti
Abbreviation for Ultimate.
Veno 
Abbreviation for Venomancer. Rarely used to refer to Orb of Venom.
VS 
Initialism for Vengeful Spirit or Void Spirit. Less commonly used to mean "versus".
Wards
Items that can be placed almost anywhere on the map and provide vision around the location for a few minutes. May also refer to the unit type of the same name.
WD 
Initialism for Witch Doctor.
Well 
The fountain from either team. Derived from its name in WC3 DotA, "Well of Eternity".
Wipe 
See teamwipe.
WK 
Initialism for Wraith King.
wp 
"Well Played". Usually said by either teams when they are winning or losing and the match is either about to end or when a player did something skillful. Usually paired up with "gg".
WR 
Initialism for Windranger.
WW 
Initialism for Winter Wyvern. It also may refer to "Wind Walk" abilities, which is derived from the common DotA name of the invisibility spells of Bounty Hunter, Clinkz, Shadow Blade and Storm, and the invisibility spells of Nyx Assassin and Invoker (which already had unique names in DotA). All 6 of them are based on the Warcraft 3 spell named "Wind Walk".
Zoning
The act of harassing an enemy hero away from the creep zone in order to prevent them from gaining experience."""

In [ ]:
feature_emb_doc = wiki_lol_doc + wiki_dota_doc + dota_slang_doc+wiki_video_doc+wiki_moba_doc
feature_emb_sents = sent_tokenize(feature_emb_doc)

In [ ]:
feature_emb_doc = remove_punctuation(feature_emb_sents)

In [ ]:
def remove_stopwords(data): 
    processed_data = []
    stop_words = sw.words()
    for sentence in data:
       tokens = word_tokenize(sentence)
       filtered_sentence = [w for w in tokens if not w in stop_words]
       processed_data.append(filtered_sentence)
    return processed_data

filtered_feature_emb_doc = remove_stopwords(feature_emb_doc)

In [ ]:
feature_embedding = FastText(sentences=filtered_feature_emb_doc,size=50, window=5, min_count=1, workers=4, sg=1)

# Pos tag

## PoS tagging

In [ ]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))

(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2, random_state = 42)

In [ ]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 2 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to tag padding
tag2index['-OOV-'] = 1  # The special value used to tag OOVs

In [ ]:
def encode_sentences(sentences):
    res = []
    for sent in sentences:
        temp = [word2index[word.lower()] if word.lower() in word2index else word2index['-OOV-'] for word in sent]
        res.append(temp)
    return res

train_sentences_encoded = encode_sentences(train_sentences)
test_sentences_encoded = encode_sentences(test_sentences)


train_post_encoded = encode_sentences(training_post)
test_post_encoded = encode_sentences(testing_post)
labels_post_encoded = encode_sentences(truelabels)

train_tags_y, test_tags_y = [], []
train_post_tags_y, test_post_tags_y = [], []

def tag_to_index(tags_list):
    res = []
    for tags in tags_list:
        temp = [tag2index[tag] if tag in tag2index else tag2index['-OOV-'] for tag in tags]
        res.append(temp)
    return res

train_tags_y = tag_to_index(train_tags)
test_tags_y = tag_to_index(test_tags)



In [ ]:
# Pad to max_length
max_length = len(max(train_sentences_encoded, key=len))

In [ ]:
max_length_post = len(max(train_post_encoded, key=len))

In [ ]:
max_length_post = len(max(train_post_encoded, key=len))
def pad_sequence(seq_list, max_length, index_dict):
    res = []
    for seq in seq_list:
        temp = seq[:]
        if len(seq)>max_length:
            res.append(temp[:max_length])
        else:
            temp += [index_dict['-PAD-']] * (max_length - len(seq))
            res.append(temp)
    return np.array(res)

train_sentences_encoded_pad = pad_sequence(train_sentences_encoded+test_sentences_encoded, max_length, word2index)
# test_sentences_encoded_pad = pad_sequence(test_sentences_encoded, max_length, word2index)
train_tags_y_pad = pad_sequence(train_tags_y+test_tags_y, max_length, tag2index)
# test_tags_y_pad = pad_sequence(test_tags_y, max_length, tag2index)


# train_post_encoded_pad = pad_sequence(train_post_encoded, max_length_post, word2index)
# test_post_encoded_pad = pad_sequence(test_post_encoded, max_length_post, word2index)
# train_post_encoded_pad = pad_sequence(train_post_encoded, max_length_post, word2index)
# labels_post_encoded_pad = pad_sequence(labels_post_encoded, max_length_post, word2index)



In [ ]:
train_data = TensorDataset(torch.from_numpy(train_sentences_encoded_pad), torch.from_numpy(train_tags_y_pad))
batch_size = 128
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True) 

In [ ]:
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)  
        self.hidden2tag = nn.Linear(hidden_dim * 2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out)    
        return tag_space

EMBEDDING_DIM = 128
HIDDEN_DIM = 256

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2index), len(tag2index)).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

from sklearn.metrics import accuracy_score

number_epochs = 20

for epoch in range(number_epochs):  
    loss_now = 0.0
    correct = 0

    for sentence,targets in train_loader:
        sentence = sentence.to(device)
        targets = targets.to(device)

        temp_batch_size = sentence.shape[0]

        model.train()
        optimizer.zero_grad()               
        tag_space = model(sentence)
        loss = loss_function(tag_space.view(-1, tag_space.shape[-1]), targets.view(-1))
        loss.backward()
        optimizer.step()

        loss_now += loss.item() * temp_batch_size
        predicted = torch.argmax(tag_space, -1)
        # Note: The training accuracy here is calculated with "PAD", which will result in a relative higher accuracy.
        correct += accuracy_score(predicted.view(-1).cpu().numpy(),targets.view(-1).cpu().numpy())*temp_batch_size

    print('Epoch: %d, training loss: %.4f, training accuracy: %.2f%%'%(epoch+1,loss_now/len(train_data),100*correct/len(train_data)))

Epoch: 1, training loss: 0.6257, training accuracy: 88.15%
Epoch: 2, training loss: 0.2779, training accuracy: 93.39%
Epoch: 3, training loss: 0.2162, training accuracy: 94.66%
Epoch: 4, training loss: 0.1724, training accuracy: 95.47%
Epoch: 5, training loss: 0.1366, training accuracy: 96.41%
Epoch: 6, training loss: 0.1097, training accuracy: 97.09%
Epoch: 7, training loss: 0.0901, training accuracy: 97.59%
Epoch: 8, training loss: 0.0750, training accuracy: 98.00%
Epoch: 9, training loss: 0.0633, training accuracy: 98.31%
Epoch: 10, training loss: 0.0540, training accuracy: 98.57%
Epoch: 11, training loss: 0.0465, training accuracy: 98.77%
Epoch: 12, training loss: 0.0404, training accuracy: 98.95%
Epoch: 13, training loss: 0.0354, training accuracy: 99.09%
Epoch: 14, training loss: 0.0313, training accuracy: 99.20%
Epoch: 15, training loss: 0.0277, training accuracy: 99.30%
Epoch: 16, training loss: 0.0247, training accuracy: 99.37%
Epoch: 17, training loss: 0.0220, training accura

In [ ]:
# torch.save(model, "POSTAGGING_model.pt")

In [ ]:
# class LSTMTagger(nn.Module):
#     def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
#         super(LSTMTagger, self).__init__()
#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)  
#         self.hidden2tag = nn.Linear(hidden_dim * 2, tagset_size)

#     def forward(self, sentence):
#         embeds = self.word_embeddings(sentence)
#         lstm_out, _ = self.lstm(embeds)
#         tag_space = self.hidden2tag(lstm_out)    
#         return tag_space
# model = torch.load("/content/POSTAGGING_model.pt")

In [ ]:
train_post_encoded_pad = pad_sequence(train_post_encoded, max_length_post, word2index)
test_post_encoded_pad = pad_sequence(test_post_encoded, max_length_post, word2index)
labels_post_encoded_pad = pad_sequence(labels_post_encoded, max_length_post, word2index)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sentence_train = torch.from_numpy(train_post_encoded_pad).to(device)
sentence_test = torch.from_numpy(test_post_encoded_pad).to(device)
testloader = torch.utils.data.DataLoader(sentence_train, batch_size=1,
                                         shuffle=False)

pridiction_train = []
with torch.no_grad():
    for data in testloader:
        tag_space = model(data)
        predicted_train = torch.argmax(tag_space, -1)
        predicted_train = predicted_train.cpu().numpy()
        pridiction_train.append(predicted_train)


testloader = torch.utils.data.DataLoader(sentence_test, batch_size=1,
                                         shuffle=False)
with torch.no_grad():
    pridiction_test = []
    for data in testloader:
        tag_space = model(data)
        predicted_test = torch.argmax(tag_space, -1)
        predicted_test = predicted_test.cpu().numpy()
        pridiction_test.append(predicted_test)

In [ ]:
sentence_true_labels = torch.from_numpy(labels_post_encoded_pad).to(device)
testloader = torch.utils.data.DataLoader(sentence_true_labels, batch_size=1,
                                         shuffle=False)
with torch.no_grad():
    pridiction_true_labels = []
    for data in testloader:
        tag_space = model(data)
        predicted_true_labels = torch.argmax(tag_space, -1)
        predicted_true_labels = predicted_true_labels.cpu().numpy()
        pridiction_true_labels.append(predicted_true_labels)

In [ ]:
pridiction_train = np.array(pridiction_train).reshape(26086,-1)
pridiction_test = np.array(pridiction_test).reshape(8706,-1)
sentence_true_labels = np.array(pridiction_true_labels).reshape(-1,57)

In [ ]:
pridiction_train

In [ ]:
# Cut off padding and get POS tags of every sentence
train_len_list = [len(s) for s in train_post_encoded]
predicted_train_list = []
pridiction_train_list = [pridiction_train[i][:train_len_list[i]] for i in range(len(pridiction_train))]

test_len_list = [len(s) for s in test_post_encoded]
predicted_test_list = []
pridiction_test_list = [pridiction_test[i][:test_len_list[i]] for i in range(len(pridiction_test))]

labels_len_list = [len(s) for s in labels_post_encoded]
predicted_labels_list = []
predicted_labels_list = [sentence_true_labels[i][:test_len_list[i]] for i in range(len(sentence_true_labels))]

In [ ]:
pos_full = pridiction_train_list + pridiction_test_list + predicted_labels_list

In [ ]:
full_data_pos = training_post + testing_post + truelabels

In [ ]:
d = {}
for i, sent in enumerate(full_data_pos):
    d.update(dict(zip(sent,pos_full[i])))

In [ ]:
emb_dim = 100
emb_table = []
for i, word in enumerate(word_list):
    if (word in sem_model.wv) and (word in feature_embedding.wv):
        emb_table.append(np.concatenate((sem_model.wv[word],feature_embedding.wv[word]),0))
    else:
        emb_table.append(np.array([0]*emb_dim))

    if word in d.keys():
        num = d[word]
        number = np.zeros(len(tag2index))
        number[num] = 1
        emb_table[i] = np.concatenate((emb_table[i], number),0)
    else:
        number = np.zeros(len(tag2index))
        emb_table[i] = np.concatenate((emb_table[i], number),0)


emb_table_ABC = np.array(emb_table)

emb_table_ABC_dim = emb_table_ABC.shape[1]
print(emb_table_ABC_dim)
# detect_zero_array(emb_table_ABC)

148


In [ ]:
emb_dim = 100
emb_table = []
for i, word in enumerate(word_list):
    if (word in sem_model.wv) and (word in feature_embedding.wv):
        emb_table.append(np.concatenate((sem_model.wv[word],feature_embedding.wv[word]),0))
    else:
        emb_table.append(np.array([0]*emb_dim))

emb_table_AB = np.array(emb_table)
emb_table_AB_dim = emb_table_AB.shape[1]
print(emb_table_AB_dim)
# detect_zero_array(emb_table_AB)

100


#Toxic Embedding

In [ ]:
import pandas as pd
with open('/content/drive/MyDrive/CONDA/data/toxic_sentences.csv', 'r') as f:
    toxic_sentences=pd.read_csv(f)

toxic=toxic_sentences[(toxic_sentences['toxic']==1) | (toxic_sentences['severe_toxic']==1)]
toxic_text=toxic.comment_text.tolist()
toxic_text=toxic_text[:1000]

import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stopwords=sw.words('english')
from nltk.tokenize import word_tokenize, sent_tokenize

toxic_preprocessed=[]
for i in toxic_text:
    i=i.lower()
    i=i.replace('\n', ' ')
    i=re.sub(r'[^a-z0-9]',' ', i)
    a=word_tokenize(i)
    text=[w for w in a if not w in stopwords and len(w)>1]
    toxic_preprocessed.append(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from gensim.models import FastText
tox_model = FastText(sentences = toxic_preprocessed, size=64, window=3, min_count=1, workers=4, sg=1, batch_words=300)
tox_emb_table, count=construct_embedding_tabel(word_dict, tox_model)

In [ ]:
tox_emb_table.shape

(12770, 64)

# Concate Embdding

In [ ]:
emb_table=np.hstack((sem_emb_table, dom_emb_table, tox_emb_table))
# emb_table=np.hstack((sem_emb_table, dom_emb_table))

In [ ]:
emb_table.shape

(12770, 192)

# Encode slot label

In [ ]:
train_data.slot_label.tolist()

In [ ]:
record_slot=[]
for i in train_data.slot_label.tolist():
    for j in i:
        if j not in record_slot:
            record_slot.append(j)
for i in valid_data.slot_label.tolist():
    for j in i:
        if j not in record_slot:
            record_slot.append(j)
record_slot.sort()

In [ ]:
record_slot

['C', 'D', 'O', 'P', 'S', 'SEPA', 'T']

In [ ]:
train_data

,lower_token,slot_label,sentence_encode
0,[wow],[O],[12390]
1,[wtf],[T],[12449]
2,"[wpe, wpe]","[O, O]","[12403, 12403]"
3,[hahaha],[O],[5045]
4,[wtf],[T],[12449]
...,...,...,...
26081,"[i, bet]","[P, O]","[5576, 1676]"
26082,"[cant, believe, it]","[O, O, P]","[2209, 1652, 5863]"
26083,[ah],[O],[823]
26084,"[dayuuuuuum, sepa, lool]","[O, SEPA, O]","[3009, 9850, 6770]"


In [ ]:
from sklearn import preprocessing
train_slot_encode=[]
valid_slot_encode=[]
label_encoder_slot = preprocessing.LabelEncoder()
label_encoder_slot.fit(record_slot)
print(label_encoder_slot.classes_)
for i in range(train_data.shape[0]):
    train_slot_encode.append(label_encoder_slot.transform(train_data.slot_label.tolist()[i]).tolist())
for j in range(valid_data.shape[0]):
    valid_slot_encode.append(label_encoder_slot.transform(valid_data.slot_label.tolist()[j]).tolist())

['C' 'D' 'O' 'P' 'S' 'SEPA' 'T']


In [ ]:
len(label_encoder_slot.classes_.tolist())

7

In [ ]:
train_data=train_data.drop(columns=['slot_encode'])
valid_data=valid_data.drop(columns=['slot_encode'])

KeyError: ignored

In [ ]:
train_data.insert(train_data.shape[-1], 'slot_encode', train_slot_encode)
valid_data.insert(valid_data.shape[-1], 'slot_encode', valid_slot_encode)

In [ ]:
train_data

,lower_token,slot_label,slot_encode
0,[wow],[O],[2]
1,[wtf],[T],[6]
2,"[wpe, wpe]","[O, O]","[2, 2]"
3,[hahaha],[O],[2]
4,[wtf],[T],[6]
...,...,...,...
31081,"[passsion, over]","[O, O]","[2, 2]"
31082,[ya],[O],[2]
31083,"[just, end, it]","[O, O, P]","[2, 2, 3]"
31084,[hahahahah],[O],[2]


In [ ]:
list(label_encoder_slot.inverse_transform([3, 2, 5, 2, 5, 2, 2, 2]))

['P', 'O', 'SEPA', 'O', 'SEPA', 'O', 'O', 'O']

# Model

In [ ]:
%%capture
!pip install pytorch-crf
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torchcrf import CRF

In [ ]:
#Batch_size=1
class BiLSTM_ATTEN_CRF(nn.Module):
    def __init__(self, lookup_table, hidden_dim, tag_nums):
        super().__init__()
        self.word_emb=nn.Embedding(lookup_table.shape[0], lookup_table.shape[1])
        self.word_emb.weight.data.copy_(torch.from_numpy(lookup_table))

        self.hidden_dim=hidden_dim
        self.lstm=nn.LSTM(lookup_table.shape[1], hidden_dim//2, bidirectional=True, batch_first=True)

        self.W_Q = nn.Linear(hidden_dim, hidden_dim, bias =False) #(B, L, H)
        self.W_K = nn.Linear(hidden_dim, hidden_dim ,bias =False)
        self.W_V = nn.Linear(hidden_dim, hidden_dim, bias =False)

        self.tag_nums=tag_nums
        self.hidden2tag = nn.Linear(hidden_dim, tag_nums)

        self.dropout = nn.Dropout(p=0.1)
        self.crf = CRF(num_tags=tag_nums , batch_first=True)

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))
    
    def attention(self, Q, K, V):
        d_k=K.size(-1)
        scores=torch.matmul(Q, K.transpose(1,2))/math.sqrt(d_k) #(B, L, L)
        distribution=F.softmax(scores, dim=-1) #(B, L, L)
        Z = torch.matmul(distribution, V) #(B, L, H)
        return Z
    
    def forward(self, sentence_encode, tag_encode=None):
        input=self.word_emb(sentence_encode)#input(B, L, H)
        output, _ = self.lstm(input, self.init_hidden()) #output(B, L, H)
        Q = self.W_Q(output)  #(B, L, H)
        K = self.W_K(output)
        V = self.W_V(output)
        atten_output = self.attention(Q, K, V) #(B, L, H)
        emissions =self.hidden2tag(atten_output) #(B, L, tag_nums)
        if tag_encode != None:
            loss=-1*(self.crf(emissions, tag_encode))
            return loss, emissions
        else:
            return emissions

# Dataloader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, mode):
        self.df = df
        self.mode = mode

    def __len__(self):
        return len(self.df)


    def __getitem__(self, index):
        sentence_encode=self.df.sentence_encode[index]

        if self.mode != 'test':
            slot_encode=self.df.slot_encode[index]
            return {
                'sentence_encode': torch.tensor(sentence_encode, dtype=torch.long),
                'slot_encode': torch.tensor(slot_encode, dtype=torch.long)
            }

        else:
            return {
                'sentence_encode': torch.tensor(sentence_encode, dtype=torch.long)
            }

In [ ]:
def BuildDataloader(df, mode, batch_size=1, shuffle=True):
    cumstom_data = CustomDataset(df, mode)
    data_loader = DataLoader(cumstom_data, batch_size=batch_size, num_workers=2, shuffle=shuffle, pin_memory=True)
    return data_loader

# Train

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
from itertools import chain

def train(model, optimizer, training_loader): 
    model.train()
    #用于累计一个epoch的loss
    slot_loss_recorder = []
    sum_loss_recorder = []

    #用于累计一个epoch的真实结果和预测结果
    slot_target_recorder = []
    slot_predition_recorder = []


    for data in tqdm(training_loader, total=len(training_loader)): #len(training_loader)=data_num/batch_size
        sentence_encode = data['sentence_encode'].to(device)
        slot_encode =data['slot_encode'].to(device) #shape:(batch_size, sequence_length)

        optimizer.zero_grad()

        #inference        
        loss, emissions = model(sentence_encode, slot_encode) #emissions.shape=(batch_size, num_labels)
        
        slot_loss_recorder.append(loss.item())

        loss.backward()

        optimizer.step()
        

        #用于计算slot的F1
        slot_target_recorder.extend(slot_encode.squeeze(dim=0).cpu().detach().numpy().tolist())
        slot_predition_recorder.extend(list(chain.from_iterable(model.crf.decode(emissions))))

    
    #计算一个epoch的平均损失，平均体现在batch_size
    slot_loss_epoch = np.average(slot_loss_recorder)

    #计算一个epoch中intent和slot的结果
    slot_name=['C', 'D', 'O', 'P', 'S', 'SEPA', 'T']
    slot_result=classification_report(slot_target_recorder, slot_predition_recorder, zero_division=0,
                                        target_names=slot_name, output_dict=True)
    slot_df=pd.DataFrame(slot_result).T
    slot_df['support'] = slot_df.support.apply(int)
    slot_df=slot_df.style.background_gradient(cmap='hot', subset=pd.IndexSlice[:slot_name[-1], :'f1-score'])


    return{
        'slot_loss': slot_loss_epoch,
        'slot_result': slot_df
        }

# Valid

In [ ]:
def validation(model, validation_loader):
    model.eval()
    #用于累计一个epoch的loss
    slot_loss_recorder=[]

    #用于累计一个epoch的真实结果和预测结果
    slot_target_recorder = []
    slot_predition_recorder = []
    
    with torch.no_grad():
        for data in tqdm(validation_loader, total=len(validation_loader)):
            sentence_encode = data['sentence_encode'].to(device)
            slot_encode =data['slot_encode'].to(device) #shape:(batch_size, sequence_length)

            loss, emissions = model(sentence_encode, slot_encode) #emissions.shape=(batch_size, num_labels)

            slot_loss_recorder.append(loss.item())

            #用于计算slot的F1
            slot_target_recorder.extend(slot_encode.squeeze(dim=0).cpu().detach().numpy().tolist())
            slot_predition_recorder.extend(list(chain.from_iterable(model.crf.decode(emissions))))


        #计算一个epoch的平均损失，平均体现在batch_size
        slot_loss_epoch = np.average(slot_loss_recorder)
        #计算一个epoch中intent和slot的结果
        slot_name=['C', 'D', 'O', 'P', 'S', 'SEPA', 'T']
        slot_result=classification_report(slot_target_recorder, slot_predition_recorder, zero_division=0, 
                                            target_names=slot_name, output_dict=True)
        slot_df=pd.DataFrame(slot_result).T
        slot_df['support'] = slot_df.support.apply(int)
        slot_df=slot_df.style.background_gradient(cmap='hot', subset=pd.IndexSlice[:slot_name[-1], :'f1-score'])
        
    return{
        'slot_loss': slot_loss_epoch,
        'slot_result': slot_df
        }

# Fit

In [ ]:
def fit(model, epochs, optimizer, training_loader, validation_loader):
    #train相关的数值记录
    train_slot_loss=[]

    #valid相关的数值记录
    valid_slot_loss=[]

    #模型名字
    model_name=model.__class__.__name__

    # early stopping相关初始参数
    best_valid_slot_loss = np.iinfo(np.int32).max
    best_f1 = np.iinfo(np.int32).min
    cumulative_descent = 0
    
    for epoch in range(epochs):
        flag=False

        #inference process
        train_outputs = train(model, optimizer, training_loader)

        #train_loss相关
        train_slot_loss.append(train_outputs['slot_loss'])

        #validation process
        valid_outputs = validation(model, validation_loader)

        #valid_loss相关
        valid_slot_loss.append(valid_outputs['slot_loss'])


        # #Early stopping
        # if best_valid_slot_loss > valid_outputs['slot_loss']:
        #     flag=True
        #     cumulative_descent = 0
        #     best_valid_slot_loss =  valid_outputs['slot_loss']
        #     torch.save(model.state_dict(), f'{model_name}_checkpoint_bestloss.pt')
        # else:
        #     cumulative_descent += 1

        #save model
        torch.save(model.state_dict(), f'{model_name}_checkpoint_epoch{epoch+1}.pt')

        

        # print(f"Epoch: {(epoch+1):02d}丨Train->    total_loss: {(train_total_loss[-1]):.4f}丨intent_loss: {(train_intent_loss[-1]):.4f}丨intent_f1: {(100*(train_intent_f1[-1])):.2f}%丨slot_loss: {(train_slot_loss[-1]):.4f}丨slot_f1: {(100*(train_slot_f1[-1])):.2f}%")
        # print(f"Epoch: {(epoch+1):02d}丨Valid->    total_loss: {(valid_total_loss[-1]):.4f}丨intent_loss: {(valid_intent_loss[-1]):.4f}丨intent_f1: {(100*(valid_intent_f1[-1])):.2f}%丨slot_loss: {(valid_slot_loss[-1]):.4f}丨slot_f1: {(100*(valid_slot_f1[-1])):.2f}%")
        print(f"Epoch: {(epoch+1):02d}丨Train->    slot_loss: {(train_slot_loss[-1]):.4f}")
        print()
        print(f'slot result:')
        display(train_outputs['slot_result'])
        print()
        print(f"Epoch: {(epoch+1):02d}丨Valid->    slot_loss: {(valid_slot_loss[-1]):.4f}")
        print(f'slot result:')
        display(valid_outputs['slot_result'])
        
        print(f'Update checkpoint: {flag}丨cumulative_descent: {cumulative_descent}')
        print('\n')
        print('=================================================================================================')
        print('\n')
        if cumulative_descent == 2: 
            break
            
    # return

# Main

In [ ]:
lookup_table=emb_table_ABC
hidden_dim=70
tag_nums=len(label_encoder_slot.classes_.tolist())
model=BiLSTM_ATTEN_CRF(lookup_table, hidden_dim, tag_nums).to(device)

train_data_loader=BuildDataloader(train_data, mode='train', shuffle=True)
valid_data_loader=BuildDataloader(valid_data, mode='valid', shuffle=True)

criterion = nn.CrossEntropyLoss()

total_epoch = 3
learning_rate = 0.01

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
train_data

,lower_token,slot_label,slot_encode
0,[wow],[O],[2]
1,[wtf],[T],[6]
2,"[wpe, wpe]","[O, O]","[2, 2]"
3,[hahaha],[O],[2]
4,[wtf],[T],[6]
...,...,...,...
31081,"[passsion, over]","[O, O]","[2, 2]"
31082,[ya],[O],[2]
31083,"[just, end, it]","[O, O, P]","[2, 2, 3]"
31084,[hahahahah],[O],[2]


In [ ]:
fit(model, total_epoch, optimizer, train_data_loader, valid_data_loader)

  0%|          | 0/26086 [00:00<?, ?it/s]

  0%|          | 0/8706 [00:00<?, ?it/s]

Epoch: 01丨Train->    slot_loss: 0.5874

slot result:


,precision,recall,f1-score,support
C,0.910589,0.843547,0.875787,4781
D,0.772834,0.518053,0.620301,1274
O,0.951534,0.992714,0.971688,56820
P,0.985973,0.931411,0.957915,11999
S,0.963563,0.924870,0.943820,10036
SEPA,0.991737,0.979267,0.985462,10418
T,0.931687,0.825611,0.875447,4295
accuracy,0.956657,0.956657,0.956657,0
macro avg,0.929702,0.859353,0.890060,99623
weighted avg,0.955992,0.956657,0.955417,99623



Epoch: 01丨Valid->    slot_loss: 0.2058
slot result:


...


Epoch: 06丨Valid->    slot_loss: 0.2573
slot result:


,precision,recall,f1-score,support
C,0.992551,0.974406,0.983395,1641
D,0.938725,0.962312,0.950372,398
O,0.990460,0.995260,0.992854,18986
P,1.000000,0.985772,0.992835,3936
S,0.995461,0.990367,0.992908,3322
SEPA,0.999445,1.000000,0.999723,3603
T,0.963896,0.963240,0.963568,1469
accuracy,0.991336,0.991336,0.991336,0
macro avg,0.982934,0.981622,0.982236,33355
weighted avg,0.991370,0.991336,0.991337,33355


Update checkpoint: False丨cumulative_descent: 0






  0%|          | 0/26086 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Main 2

In [ ]:
fit(model, total_epoch, optimizer, train_data_loader, valid_data_loader)

  0%|          | 0/31086 [00:00<?, ?it/s]

  0%|          | 0/3706 [00:00<?, ?it/s]

Epoch: 01丨Train->    slot_loss: 1.6196

slot result:


,precision,recall,f1-score,support
C,0.725403,0.456453,0.560326,5718
D,0.769953,0.109699,0.192037,1495
O,0.854000,0.981606,0.913368,67741
P,0.966914,0.758594,0.850179,14254
S,0.865539,0.783963,0.822734,11947
SEPA,0.946523,0.905742,0.925683,12487
T,0.835660,0.512751,0.635541,5137
accuracy,0.870457,0.870457,0.870457,0
macro avg,0.851999,0.644115,0.699981,118779
weighted avg,0.870396,0.870457,0.859874,118779



Epoch: 01丨Valid->    slot_loss: 0.3207
slot result:


...

Epoch: 03丨Train->    slot_loss: 0.0671

slot result:


,precision,recall,f1-score,support
C,0.989285,0.984960,0.987118,5718
D,0.929326,0.941137,0.935194,1495
O,0.996554,0.998893,0.997722,67741
P,0.998665,0.997264,0.997964,14254
S,0.998149,0.993053,0.995594,11947
SEPA,1.000000,1.000000,1.000000,12487
T,0.990918,0.977029,0.983925,5137
accuracy,0.995883,0.995883,0.995883,0
macro avg,0.986128,0.984619,0.985360,118779
weighted avg,0.995890,0.995883,0.995882,118779



Epoch: 03丨Valid->    slot_loss: 0.1224
slot result:


,precision,recall,f1-score,support
C,0.997076,0.968750,0.982709,704
D,0.934066,0.960452,0.947075,177
O,0.993583,0.998388,0.995980,8065
P,1.000000,0.998215,0.999107,1681
S,1.000000,0.993622,0.996801,1411
SEPA,1.000000,1.000000,1.000000,1534
T,0.988618,0.969697,0.979066,627
accuracy,0.994859,0.994859,0.994859,0
macro avg,0.987620,0.984161,0.985820,14199
weighted avg,0.994886,0.994859,0.994852,14199


Update checkpoint: False丨cumulative_descent: 0






In [ ]:
total_epoch = 2
learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
total_epoch = 2
learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
fit(model, total_epoch, optimizer, train_data_loader, valid_data_loader)

  0%|          | 0/31086 [00:00<?, ?it/s]

  0%|          | 0/3706 [00:00<?, ?it/s]

Epoch: 01丨Train->    slot_loss: 0.0226

slot result:


,precision,recall,f1-score,support
C,0.996497,0.994928,0.995712,5718
D,0.972332,0.987291,0.979754,1495
O,0.999026,0.999601,0.999314,67741
P,0.999579,0.999158,0.999368,14254
S,0.999329,0.996819,0.998072,11947
SEPA,0.999920,1.000000,0.999960,12487
T,0.994726,0.991240,0.992980,5137
accuracy,0.998569,0.998569,0.998569,0
macro avg,0.994487,0.995577,0.995023,118779
weighted avg,0.998573,0.998569,0.998570,118779



Epoch: 01丨Valid->    slot_loss: 0.1187
slot result:


...

Epoch: 02丨Train->    slot_loss: 0.0176

slot result:


,precision,recall,f1-score,support
C,0.997023,0.995628,0.996325,5718
D,0.971729,0.988629,0.980106,1495
O,0.999233,0.999793,0.999513,67741
P,0.999789,0.999369,0.999579,14254
S,0.999580,0.997070,0.998324,11947
SEPA,0.999920,1.000000,0.999960,12487
T,0.997850,0.993771,0.995806,5137
accuracy,0.998889,0.998889,0.998889,0
macro avg,0.995018,0.996323,0.995659,118779
weighted avg,0.998894,0.998889,0.998890,118779



Epoch: 02丨Valid->    slot_loss: 0.1223
slot result:


,precision,recall,f1-score,support
C,0.992775,0.975852,0.984241,704
D,0.965909,0.960452,0.963173,177
O,0.993953,0.998636,0.996289,8065
P,0.999404,0.998215,0.998810,1681
S,1.000000,0.992913,0.996444,1411
SEPA,1.000000,1.000000,1.000000,1534
T,0.990228,0.969697,0.979855,627
accuracy,0.995281,0.995281,0.995281,0
macro avg,0.991753,0.985109,0.988402,14199
weighted avg,0.995280,0.995281,0.995268,14199


Update checkpoint: False丨cumulative_descent: 0






# Main 3

In [ ]:
lookup_table=emb_table_ABC
hidden_dim=148
tag_nums=len(label_encoder_slot.classes_.tolist())
model=BiLSTM_ATTEN_CRF(lookup_table, hidden_dim, tag_nums).to(device)

train_data_loader=BuildDataloader(train_data, mode='train', shuffle=True)
valid_data_loader=BuildDataloader(valid_data, mode='valid', shuffle=True)

criterion = nn.CrossEntropyLoss()

total_epoch = 3
learning_rate = 0.01

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
fit(model, total_epoch, optimizer, train_data_loader, valid_data_loader)

  0%|          | 0/31086 [00:00<?, ?it/s]

  0%|          | 0/3706 [00:00<?, ?it/s]

Epoch: 01丨Train->    slot_loss: 1.7342

slot result:


,precision,recall,f1-score,support
C,0.708153,0.429871,0.534987,5718
D,0.764151,0.054181,0.101187,1495
O,0.842482,0.977148,0.904832,67741
P,0.957111,0.743651,0.836985,14254
S,0.862315,0.757512,0.806523,11947
SEPA,0.935166,0.882518,0.908080,12487
T,0.813358,0.519175,0.633793,5137
accuracy,0.859319,0.859319,0.859319,0
macro avg,0.840391,0.623436,0.675198,118779
weighted avg,0.859265,0.859319,0.847502,118779



Epoch: 01丨Valid->    slot_loss: 0.4155
slot result:


...

Epoch: 03丨Train->    slot_loss: 0.0794

slot result:


,precision,recall,f1-score,support
C,0.982340,0.982511,0.982425,5718
D,0.910438,0.904348,0.907383,1495
O,0.996452,0.999203,0.997826,67741
P,0.998876,0.997615,0.998245,14254
S,0.997473,0.991379,0.994417,11947
SEPA,0.999680,1.000000,0.999840,12487
T,0.988704,0.971189,0.979868,5137
accuracy,0.995100,0.995100,0.995100,0
macro avg,0.981995,0.978035,0.980001,118779
weighted avg,0.995088,0.995100,0.995088,118779



Epoch: 03丨Valid->    slot_loss: 0.1568
slot result:


,precision,recall,f1-score,support
C,0.978448,0.967330,0.972857,704
D,0.888889,0.949153,0.918033,177
O,0.993678,0.993924,0.993801,8065
P,0.987633,0.997620,0.992601,1681
S,0.999287,0.992913,0.996090,1411
SEPA,1.000000,1.000000,1.000000,1534
T,0.990212,0.968102,0.979032,627
accuracy,0.991901,0.991901,0.991901,0
macro avg,0.976878,0.981292,0.978916,14199
weighted avg,0.991988,0.991901,0.991921,14199


Update checkpoint: False丨cumulative_descent: 0






In [ ]:
total_epoch = 3
learning_rate = 0.008
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
fit(model, total_epoch, optimizer, train_data_loader, valid_data_loader)

  0%|          | 0/31086 [00:00<?, ?it/s]

  0%|          | 0/3706 [00:00<?, ?it/s]

Epoch: 01丨Train->    slot_loss: 0.0348

slot result:


,precision,recall,f1-score,support
C,0.993176,0.992655,0.992915,5718
D,0.946711,0.962542,0.954561,1495
O,0.998569,0.999469,0.999019,67741
P,0.999438,0.998597,0.999017,14254
S,0.998572,0.994810,0.996687,11947
SEPA,0.999920,1.000000,0.999960,12487
T,0.993738,0.988515,0.991119,5137
accuracy,0.997685,0.997685,0.997685,0
macro avg,0.990018,0.990941,0.990468,118779
weighted avg,0.997695,0.997685,0.997688,118779



Epoch: 01丨Valid->    slot_loss: 0.1208
slot result:


...

Epoch: 03丨Train->    slot_loss: 0.0106

slot result:


,precision,recall,f1-score,support
C,0.997903,0.998776,0.998339,5718
D,0.979470,0.989298,0.984359,1495
O,0.999675,0.999882,0.999779,67741
P,0.999789,0.999509,0.999649,14254
S,0.999413,0.997405,0.998408,11947
SEPA,1.000000,1.000000,1.000000,12487
T,0.997856,0.996691,0.997273,5137
accuracy,0.999276,0.999276,0.999276,0
macro avg,0.996301,0.997366,0.996830,118779
weighted avg,0.999278,0.999276,0.999277,118779



Epoch: 03丨Valid->    slot_loss: 0.1438
slot result:


,precision,recall,f1-score,support
C,0.987088,0.977273,0.982156,704
D,0.988304,0.954802,0.971264,177
O,0.994198,0.998636,0.996412,8065
P,1.000000,0.998215,0.999107,1681
S,0.997866,0.994330,0.996095,1411
SEPA,1.000000,1.000000,1.000000,1534
T,0.995098,0.971292,0.983051,627
accuracy,0.995493,0.995493,0.995493,0
macro avg,0.994651,0.984936,0.989726,14199
weighted avg,0.995490,0.995493,0.995477,14199


Update checkpoint: False丨cumulative_descent: 0






# Test

In [ ]:
def test(model, test_loader):
    model.eval()

    #用于累计一个epoch的预测结果
    slot_predition_recorder = []
    
    with torch.no_grad():
        for data in tqdm(test_loader, total=len(test_loader)):
            sentence_encode = data['sentence_encode'].to(device)

            emissions = model(sentence_encode) #emissions.shape=(batch_size, num_labels)

            slot_predition_recorder.extend(model.crf.decode(emissions))

    return{
        'slot_predition': slot_predition_recorder
        }

In [ ]:
lookup_table=emb_table_ABC
hidden_dim=148
tag_nums=len(label_encoder_slot.classes_.tolist())

best_model=BiLSTM_ATTEN_CRF(lookup_table, 148, tag_nums).to(device)
best_model.load_state_dict(torch.load('/content/BiLSTM_ATTEN_CRF_checkpoint_epoch3.pt'))

<All keys matched successfully>

In [ ]:
test_data_loader=BuildDataloader(test_data, mode='test', batch_size=1, shuffle=False)

In [ ]:
results=test(best_model, test_data_loader)

  0%|          | 0/8703 [00:00<?, ?it/s]

In [ ]:
slot_answer=[]
for slot in results['slot_predition']:
    slot_answer.append(list(label_encoder_slot.inverse_transform(slot)))

In [ ]:
with open('/content/slot.txt', mode='r') as f:
    intent_answer = f.read()

merge_results=[]
intent=intent_answer.split('\n')
for j in range(8703):
    merge_results.append(intent[j][0]+' '+' '.join(slot_answer[j]))

with open('qq4.txt', mode='a') as f:
    for i in merge_results:
        f.write(i)
        f.write('\n')

In [ ]:
slot_answer

In [ ]:
with open('slot_answer.txt', mode='a') as f:
    for i in slot_answer:
        f.write(' '.join(i))
        f.write('\n')

In [ ]:
slot_answer